In [2]:
import os
import pandas as pd
import numpy as np

analog_root = "Analog Data"
pressure_results = []

def voltage_to_pressure(vout):
    return (vout / 5 - 0.04) / 0.009

csv_files = sorted([
    f for f in os.listdir(analog_root)
    if f.endswith('.csv') and not f.startswith('.')
])

for fname in csv_files:
    try:
        freq, pwm = fname.replace('.csv', '').split('_')
        freq = int(freq)
        pwm = int(pwm)
    except:
        print(f"Skipping malformed filename: {fname}")
        continue

    path = os.path.join(analog_root, fname)
    try:
        df = pd.read_csv(path)
        if 'Channel_3' not in df or 'Channel_4' not in df or 'Time' not in df:
            print(f"Missing expected columns in {fname}")
            continue

        # Detect where camera trigger (Channel_4) goes high
        camera_on = df['Channel_4'] >= 4.5

        if not camera_on.any():
            print(f"No camera trigger found in {fname}")
            continue

        # First time the camera signal goes high
        start_idx = camera_on.idxmax()
        start_time = df.loc[start_idx, 'Time']
        end_time = start_time + 0.5  # 0.5s = 500 frames @ 1000fps

        # Get only the rows within that time window
        recording = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]

        if len(recording) < 10:
            print(f"⚠️ Too few samples in {fname} after sync")
            continue

        vout = recording['Channel_3'].values
        pressure = voltage_to_pressure(vout)

        pressure_results.append({
            "frequency": freq,
            "pump_pwm": pwm,
            "mean_pressure": np.mean(pressure),
            "max_pressure": np.max(pressure),
            "min_pressure": np.min(pressure),
            "amplitude_pressure": np.max(pressure) - np.min(pressure)
        })

    except Exception as e:
        print(f"Error in {fname}: {e}")

# Final DataFrame
pressure_df = pd.DataFrame(pressure_results)
pressure_df = pressure_df.sort_values(by=["frequency", "pump_pwm"]).reset_index(drop=True)

Skipping malformed filename: 020_200_2.csv


In [3]:
pressure_df

,frequency,pump_pwm,mean_pressure,max_pressure,min_pressure,amplitude_pressure
0,0,0,0.627922,0.716417,0.560438,0.155979
1,0,200,43.899542,45.407715,42.023655,3.384060
2,5,200,14.231872,31.959635,0.668945,31.290690
3,10,50,1.918596,5.226237,0.167101,5.059136
4,10,70,3.351606,6.880968,0.546875,6.334093
5,10,200,12.184213,24.120009,0.892741,23.227268
6,15,200,11.702800,21.380208,1.815050,19.565158
7,20,50,1.894000,5.049913,0.587565,4.462348
8,20,60,2.354666,5.578885,0.655382,4.923503
9,20,70,2.964504,5.836589,0.662164,5.174425


In [ ]:
# plot the pressure vs frequency

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=pressure_df, x='frequency', y='mean_pressure', hue='pump_pwm', marker='o')
plt.title('Mean Pressure vs Frequency')
plt.xlabel('Frequency (Hz)')

In [5]:
# for "000_000.csv", how many samples of data are there when the camera trigger goes high? calculate the number only
camera_trigger_file = '000_000.csv'
camera_trigger_path = os.path.join(analog_root, camera_trigger_file)
camera_trigger_df = pd.read_csv(camera_trigger_path)
camera_trigger_on = camera_trigger_df['Channel_4'] >= 4.5
camera_trigger_start_idx = camera_trigger_on.idxmax()
camera_trigger_start_time = camera_trigger_df.loc[camera_trigger_start_idx, 'Time']
camera_trigger_end_time = camera_trigger_start_time + 1  # 0.5s = 500 frames @ 1000fps
camera_trigger_recording = camera_trigger_df[
    (camera_trigger_df['Time'] >= camera_trigger_start_time) &
    (camera_trigger_df['Time'] <= camera_trigger_end_time)
]
camera_trigger_sample_count = len(camera_trigger_recording)
print(f"Number of samples in {camera_trigger_file} after sync: {camera_trigger_sample_count}")

Number of samples in 000_000.csv after sync: 507
